In [6]:
# Import library
from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.preprocessing import MinMaxScaler,StandardScaler
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.feature_extraction.text import TfidfVectorizer

import re
# from sklearn.preprocessing import OneHotEncoder
# from nltk.util import ngrams
# from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np
import simplejson as json
from pprint import pprint
from tqdm import tqdm

In [7]:
df_track_info_complete = pd.read_pickle('../data/complete_feature.pkl')
df_track_info_complete

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.000000,0.912210,0.813,0.363636,0.842733,0.0,0.125649,0.031225,0.006977,0.0471,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.000006,0.781029,0.838,0.454545,0.893573,0.0,0.118380,0.025000,0.025025,0.2420,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.000012,0.670030,0.758,0.181818,0.851050,0.0,0.218069,0.002390,0.000000,0.0598,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
spotify:track:1AWQoqb9bSvzTjaLralEkT,0.000018,0.900101,0.714,0.363636,0.859462,0.0,0.146417,0.201807,0.000234,0.0521,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
spotify:track:1lzr43nnXAijIGYnCT8M8H,0.000024,0.860747,0.606,0.000000,0.882707,1.0,0.074039,0.056325,0.000000,0.3130,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:4cKPzfOUIZWXhu3GDnESjL,0.999976,0.556004,0.822,1.000000,0.844119,0.0,0.063759,0.001546,0.000000,0.2150,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
spotify:track:0VICzaRdctD8XGfDstBjQY,0.999982,0.484359,0.761,0.545455,0.882197,1.0,0.038110,0.120482,0.000000,0.2960,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
spotify:track:0B4YuevrDNhh71edZrFiOs,0.999988,0.405651,0.767,1.000000,0.848899,0.0,0.177570,0.000078,0.000003,0.3930,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [8]:
with open("../data/train_playlist.pkl","rb") as f:
    df_playlists_train = pd.read_pickle(f)
df_playlists_train

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
9254,NOLA,false,9254,1507939200,66,61,1,"[spotify:track:0nMqu5vN44GNv8UOQxmBJK, spotify...",17,14752318,56,NaN
1561,H2O,false,1561,1385596800,43,43,1,"[spotify:track:2ag8U9ccVmjBjia5qNZPDH, spotify...",17,9915452,42,NaN
1670,THE MIX,false,1670,1472083200,62,58,2,"[spotify:track:11bD1JtSjlIgKgZG2134DZ, spotify...",7,15286108,47,NaN
6087,Peaceful,false,6087,1507593600,106,98,1,"[spotify:track:152lZdxL1OR0ZMW6KquMif, spotify...",24,24124550,81,NaN
6669,New,false,6669,1489449600,16,15,1,"[spotify:track:0VYCUWNFIYLIOQqu3bFzs0, spotify...",11,4071423,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,Country,false,5734,1451779200,223,152,3,"[spotify:track:1mJ05BN0So26a14iib85aI, spotify...",4,48199560,70,NaN
5191,Folk,false,5191,1508716800,107,51,1,"[spotify:track:21pL95f63XT8Mezt8qPOtq, spotify...",15,21989208,43,NaN
5390,jams,false,5390,1509408000,101,79,1,"[spotify:track:3DXncPQOG4VBw3QHh3S817, spotify...",34,21776300,61,NaN
860,work,false,860,1493424000,50,43,1,"[spotify:track:4SibKHGH0Nql4sZwOweCh2, spotify...",17,12286033,39,NaN


In [9]:
with open("../data/test_playlist.pkl","rb") as f:
    df_playlists_test = pd.read_pickle(f)
df_playlists_test

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
6252,Gym,false,6252,1486598400,30,28,1,"[spotify:track:3CzOoYfw7mj5jMHGsJp9TG, spotify...",10,6771523,19,NaN
4684,BOSSanova,false,4684,1498176000,20,13,1,"[spotify:track:0wrx7YMUJXIcThebT8XDz5, spotify...",5,4473182,10,NaN
1731,Disney,false,1731,1502496000,55,27,1,"[spotify:track:4T9Jpft8jvZAP57aE3VdCt, spotify...",3,10028526,39,NaN
4742,Trap,false,4742,1487980800,30,30,2,"[spotify:track:5c1Miz53kZbsJFoa6RvS99, spotify...",4,6434496,27,NaN
4521,Jesus,false,4521,1482537600,80,53,1,"[spotify:track:1wRZLcT8k5oQrf34dRBQb4, spotify...",16,24548027,43,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6412,driving,false,6412,1469577600,45,41,1,"[spotify:track:3KD51YetMdgdPRSF42E35w, spotify...",4,10615480,37,NaN
8285,Smooth rock,false,8285,1502841600,41,22,1,"[spotify:track:48UPSzbZjgc449aqz8bxox, spotify...",5,10711921,14,NaN
7853,heavy,false,7853,1440374400,17,16,1,"[spotify:track:2yuSLHwPoOoB1zDuXnQnqZ, spotify...",4,5568780,7,NaN
1095,Pop en español,false,1095,1485820800,86,53,1,"[spotify:track:5vNocq26ujTMI4L8bECuGk, spotify...",43,20344685,32,NaN


In [10]:
# df_playlists_songs = df_playlists.apply(get_track_id_in_playlist,axis=1).to_frame(name='tracks')
# df_playlists_songs.set_index(df_playlists['pid'])
# df_playlists_songs

## summarize the feature of all music within a playlist

In [11]:
def get_features_summary(x):
    temp = df_track_info_complete.loc[x['tracks'],:].mean(axis=0)
    # temp['id'] = x['pid']
    return temp

In [12]:
df_playlists_scores_train = df_playlists_train.apply(get_features_summary,axis=1)
df_playlists_scores_train

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
9254,0.459590,0.588493,0.485935,0.490358,0.809167,0.696970,0.079184,0.415698,0.198171,0.162512,...,0.000000,0.030303,0.772727,0.196970,0.242424,0.757576,0.000000,0.075758,0.772727,0.151515
1561,0.197246,0.602281,0.706744,0.471459,0.841710,0.534884,0.092115,0.155381,0.111217,0.194951,...,0.000000,0.023256,0.813953,0.162791,0.186047,0.813953,0.000000,0.093023,0.744186,0.162791
1670,0.034477,0.550243,0.739097,0.469208,0.865280,0.790323,0.041195,0.111931,0.004214,0.185289,...,0.000000,0.112903,0.693548,0.193548,0.258065,0.741935,0.000000,0.080645,0.725806,0.193548
6087,0.494865,0.579822,0.398659,0.475129,0.799889,0.509434,0.079043,0.544473,0.051560,0.173996,...,0.000000,0.075472,0.764151,0.160377,0.254717,0.745283,0.000000,0.075472,0.735849,0.188679
6669,0.273984,0.490792,0.584813,0.568182,0.842568,0.812500,0.044587,0.343547,0.000005,0.164200,...,0.000000,0.062500,0.687500,0.250000,0.312500,0.687500,0.000000,0.062500,0.750000,0.187500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,0.139292,0.591073,0.697009,0.487566,0.857117,0.910314,0.038862,0.192782,0.001052,0.164770,...,0.000000,0.094170,0.726457,0.179372,0.206278,0.789238,0.004484,0.031390,0.762332,0.206278
5191,0.468837,0.364377,0.334669,0.494477,0.800581,0.747664,0.050050,0.742571,0.005701,0.152811,...,0.018692,0.037383,0.785047,0.177570,0.093458,0.906542,0.000000,0.018692,0.906542,0.074766
5390,0.063422,0.761557,0.616248,0.397840,0.849524,0.623762,0.166262,0.137072,0.002300,0.167135,...,0.000000,0.128713,0.762376,0.108911,0.217822,0.782178,0.000000,0.049505,0.772277,0.178218
860,0.151860,0.552876,0.616540,0.405455,0.820614,0.600000,0.045695,0.311265,0.060310,0.130314,...,0.000000,0.060000,0.820000,0.120000,0.180000,0.820000,0.000000,0.040000,0.800000,0.160000


In [13]:
df_playlists_scores_test = df_playlists_test.apply(get_features_summary,axis=1)
df_playlists_scores_test

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
6252,0.077246,0.543290,0.789533,0.457576,0.872237,0.600000,0.098034,0.064792,0.002102,0.240347,...,0.000000,0.200000,0.733333,0.066667,0.366667,0.633333,0.0,0.100000,0.633333,0.266667
4684,0.523926,0.635318,0.587550,0.495455,0.794836,0.500000,0.054491,0.620181,0.065425,0.186750,...,0.000000,0.000000,1.000000,0.000000,0.100000,0.900000,0.0,0.000000,0.900000,0.100000
1731,0.121074,0.589230,0.537744,0.449587,0.792178,0.727273,0.091453,0.478968,0.004587,0.215700,...,0.000000,0.109091,0.763636,0.127273,0.200000,0.800000,0.0,0.054545,0.600000,0.345455
4742,0.362254,0.573697,0.772233,0.500000,0.873247,0.333333,0.100571,0.079371,0.036983,0.165797,...,0.000000,0.066667,0.633333,0.300000,0.366667,0.633333,0.0,0.100000,0.566667,0.333333
4521,0.263547,0.441662,0.602750,0.418182,0.843779,0.887500,0.038797,0.182700,0.006836,0.181934,...,0.000000,0.037500,0.650000,0.312500,0.237500,0.762500,0.0,0.000000,0.625000,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6412,0.310412,0.534612,0.695822,0.503030,0.849440,0.600000,0.071358,0.217239,0.108603,0.202136,...,0.022222,0.133333,0.711111,0.155556,0.200000,0.800000,0.0,0.044444,0.777778,0.177778
8285,0.490583,0.538308,0.776244,0.405765,0.870631,0.780488,0.047038,0.058120,0.009955,0.171754,...,0.000000,0.048780,0.878049,0.073171,0.146341,0.853659,0.0,0.048780,0.804878,0.146341
7853,0.384468,0.477771,0.885059,0.433155,0.848581,0.411765,0.095474,0.001254,0.145258,0.182135,...,0.000000,0.058824,0.764706,0.176471,0.294118,0.705882,0.0,0.176471,0.705882,0.117647
1095,0.175963,0.555511,0.568151,0.470402,0.854520,0.860465,0.047356,0.368462,0.000055,0.154480,...,0.000000,0.000000,0.976744,0.023256,0.116279,0.883721,0.0,0.000000,0.918605,0.081395


## Calculate Cos Similarity

### Recommend Playlists by single track

suppose the user had listend one song and we need to give him a recommendation of a playlist

In [14]:
one_song_feature = df_track_info_complete.iloc[0,:]
one_song_feature

0     0.000000
1     0.912210
2     0.813000
3     0.363636
4     0.842733
5     0.000000
6     0.125649
7     0.031225
8     0.006977
9     0.047100
10    0.810000
11    0.532108
12    0.042414
13    0.000000
14    1.000000
15    0.000000
16    0.000000
17    1.000000
18    0.000000
19    0.000000
20    1.000000
21    0.000000
22    0.000000
23    1.000000
24    0.000000
Name: spotify:track:0UaMYEvWZi0ZqiDOoHU3YI, dtype: float64

In [16]:
cosine_similarity(df_playlists_scores_train,one_song_feature.values.reshape(1,-1))

array([[0.88065652],
       [0.94252913],
       [0.88945136],
       ...,
       [0.93491814],
       [0.93455718],
       [0.931918  ]])

In [17]:
def recommend_playlist_by_feature(df_playlists_scores,features,df_playlists,n=10):
    df_recommendation = df_playlists.copy()
    df_recommendation['cos_sim'] = cosine_similarity(df_playlists_scores,features.values.reshape(1,-1))
    return df_recommendation.sort_values('cos_sim',ascending=False).head(n)

In [18]:
recommend_playlist_by_feature(df_playlists_scores_train,one_song_feature,df_playlists_train)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description,cos_sim
425,Phone,false,425,1507593600,10,10,1,"[spotify:track:6MRuNaMaH65vGyek33fXMV, spotify...",3,2339579,7,NaN,0.988541
4815,J Balvin,false,4815,1500076800,17,9,1,"[spotify:track:7syG2WcnpcoJvxSZT9VbAa, spotify...",6,3539594,5,NaN,0.985485
313,Spanish,false,313,1501804800,6,4,1,"[spotify:track:2mjqNvy3WT62V1mVZjK1nQ, spotify...",4,1212478,4,NaN,0.984462
6488,favorite,false,6488,1508457600,17,17,1,"[spotify:track:152lZdxL1OR0ZMW6KquMif, spotify...",7,3514090,17,NaN,0.984095
1050,Bhangra,false,1050,1460764800,84,72,4,"[spotify:track:53rlGWkMhyyAsgz8EcX5Yg, spotify...",23,21808612,62,NaN,0.981201
6370,💙💙,false,6370,1505952000,36,32,1,"[spotify:track:6mICuAdrwEjh6Y6lroV2Kg, spotify...",8,7856445,23,NaN,0.980650
6382,Mam,false,6382,1506643200,18,17,2,"[spotify:track:2rb5MvYT7ZIxbKW5hfcHx8, spotify...",6,3843518,12,NaN,0.979721
3734,Spanish music,false,3734,1508457600,14,13,1,"[spotify:track:0qYTZCo5Bwh1nsUFGZP3zn, spotify...",5,3138298,11,NaN,0.978807
5707,Oldies,false,5707,1508457600,7,7,1,"[spotify:track:2YegxR5As7BeQuVp2U6pek, spotify...",4,1665678,7,NaN,0.977732
1736,Reggaeton,false,1736,1401148800,40,32,5,"[spotify:track:0mRil0NkvMzXlp72oSnSDv, spotify...",3,8955468,20,NaN,0.977021


### Recommend Playlists by multiple tracks

Suppose we have a playlist and we want to recommend other playlists similar to this one

In [19]:
mimic_play_list = df_track_info_complete.iloc[[10,30,55],:]

In [20]:
recommend_playlist_by_feature(df_playlists_scores_train,mimic_play_list.mean(axis=0),df_playlists_train)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description,cos_sim
746,EBM,false,746,1507766400,21,15,1,"[spotify:track:34vs8Nv1UbY63eWgMR1Tb2, spotify...",12,5825144,12,NaN,0.982877
6750,Sleep,false,6750,1455062400,33,32,1,"[spotify:track:0wzABO1igQsSy8cQ7dIeHK, spotify...",4,8080274,26,NaN,0.981952
4980,workout,false,4980,1442534400,7,7,1,"[spotify:track:5anCkDvJ17aznvK5TED5uo, spotify...",3,1897303,6,NaN,0.977384
3913,summer,false,3913,1495152000,12,8,1,"[spotify:track:7qiZfU4dY1lWllzX7mPBI3, spotify...",3,2698524,5,NaN,0.977222
718,Swing,false,718,1494460800,24,11,1,"[spotify:track:5H0lDi9fltD8t8z6IFT34M, spotify...",3,5718853,6,NaN,0.975678
1948,Rock,false,1948,1446681600,18,9,1,"[spotify:track:5UZYlzL88CUTsyxNIm44LF, spotify...",6,4278631,6,NaN,0.974361
1837,Chill,false,1837,1504828800,10,9,3,"[spotify:track:5rLi8B8qgk6qThwRnKHW2P, spotify...",7,2257804,9,NaN,0.974066
1290,The good stuff,false,1290,1485561600,8,7,1,"[spotify:track:1N31Wy2PWz5HvCzMbff3mO, spotify...",2,2205557,5,NaN,0.973751
2283,amor,false,2283,1446940800,19,16,1,"[spotify:track:6IE47jpPeatF2Iay7GZtEc, spotify...",3,3854221,14,NaN,0.972923
9641,Running,false,9641,1482796800,45,34,1,"[spotify:track:2TfSHkHiFO4gRztVIkggkE, spotify...",2,10461300,22,NaN,0.972918


### Recommend tracks by single playlist

In [21]:
with open('../data/track_info_with_artist_album.pkl','rb') as f:
    df_track_info_with_artist_album = pd.read_pickle(f)
df_track_info_with_artist_album

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_name,track_uri,artist_uri,track_name,album_uri,album_name
uri,,,,,,,,,,,,,,,,,,
spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,125.461,226864,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,The Cookbook
spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,143.040,198800,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,In The Zone
spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,99.259,235933,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,Dangerously In Love (Alben für die Ewigkeit)
spotify:track:1AWQoqb9bSvzTjaLralEkT,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,100.972,267267,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justified
spotify:track:1lzr43nnXAijIGYnCT8M8H,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,94.759,227600,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Hot Shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:4cKPzfOUIZWXhu3GDnESjL,0.551,0.822,11,-7.018,0,0.0614,0.001540,0.000000,0.2150,0.360,127.043,265039,NateWantsToBattle,spotify:track:4cKPzfOUIZWXhu3GDnESjL,spotify:artist:0Vb15td3iKkAzdGD5Sj9Ky,Reluctant Heroes,spotify:album:05XnUTo4HrVy5eLzqcD0lg,Reluctant Heroes
spotify:track:0VICzaRdctD8XGfDstBjQY,0.480,0.761,6,-4.628,1,0.0367,0.120000,0.000000,0.2960,0.359,160.020,368767,Dima Lancaster,spotify:track:0VICzaRdctD8XGfDstBjQY,spotify:artist:7tSqWbPIUCl6bTbPoxLXGn,Your Lie in April - Medley,spotify:album:4CKBpgzwIL7NPJrOvltCHj,Your Lie in April - Medley
spotify:track:0B4YuevrDNhh71edZrFiOs,0.402,0.767,11,-6.718,0,0.1710,0.000077,0.000003,0.3930,0.558,178.293,179267,Set It Off,spotify:track:0B4YuevrDNhh71edZrFiOs,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,Uncontainable,spotify:album:0jGFC44cRkjE2b21aMEbIo,Upside Down


In [22]:
def recommend_track_by_feature(features,df_track_info,n=10):
    df_recommendation = df_track_info.copy()
    df_recommendation['cos_sim'] = cosine_similarity(df_track_info_complete,features.values.reshape(1,-1))
    return df_recommendation.sort_values('cos_sim',ascending=False).head(n)

Select single playlist

In [23]:
single_playlist_feature = df_playlists_scores_train.iloc[0,:]
single_playlist_feature

0     0.459590
1     0.588493
2     0.485935
3     0.490358
4     0.809167
5     0.696970
6     0.079184
7     0.415698
8     0.198171
9     0.162512
10    0.427197
11    0.517427
12    0.041781
13    0.196970
14    0.803030
15    0.000000
16    0.030303
17    0.772727
18    0.196970
19    0.242424
20    0.757576
21    0.000000
22    0.075758
23    0.772727
24    0.151515
Name: 9254, dtype: float64

Recommend multiple songs based on the playlist

In [24]:
recommend_track_by_feature(single_playlist_feature,df_track_info_with_artist_album)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_name,track_uri,artist_uri,track_name,album_uri,album_name,cos_sim
uri,,,,,,,,,,,,,,,,,,,
spotify:track:0sIwSqvIVSEsTD1SBJi8kJ,0.674,0.567,7,-5.671,1,0.0530,0.414,0.000000,0.1740,0.580,158.092,212000,Grupo Limite,spotify:track:0sIwSqvIVSEsTD1SBJi8kJ,spotify:artist:2qWNYueLfc979iEdLOIK2C,Ya No Me Importas,spotify:album:1O50KpV6dUvOLMhnmzdLU3,Soy Así,0.977142
spotify:track:1EjPQ7Cv0JbGEAh3Zi8AEA,0.765,0.452,7,-6.919,1,0.0329,0.444,0.435000,0.1050,0.600,139.314,378987,Willy Mason,spotify:track:1EjPQ7Cv0JbGEAh3Zi8AEA,spotify:artist:2uYZJqMXH5Q6xvPU6BPySL,Restless Fugitive,spotify:album:1a04iBNVLowvioligAF1dq,Carry On,0.976483
spotify:track:7z9SLVmHZbHomEl7jEKUDs,0.632,0.510,7,-9.141,1,0.0282,0.599,0.149000,0.1100,0.550,118.997,237480,Leighton Meester,spotify:track:7z9SLVmHZbHomEl7jEKUDs,spotify:artist:481VlDdXZAIRxnHyywNbXn,Blue Afternoon,spotify:album:2zto9vbZN0D67qlkhx5p8X,Heartstrings,0.976287
spotify:track:7IcrunsYx5Y9iu2qlX1qpH,0.699,0.582,7,-4.535,1,0.0498,0.321,0.000000,0.2450,0.478,143.961,250627,Noel Schajris,spotify:track:7IcrunsYx5Y9iu2qlX1qpH,spotify:artist:2uEjXyVi0SA1HPpj3zquXh,No Te Pertenece,spotify:album:1RcWrmTxhRZNTpWsViCp6u,Verte Nacer,0.975722
spotify:track:25vlrCfkwsGx75ETowB42N,0.795,0.643,7,-5.423,1,0.0333,0.618,0.003040,0.1200,0.464,129.968,261827,fakuta,spotify:track:25vlrCfkwsGx75ETowB42N,spotify:artist:5qW2pOdPhEkkBlsUjcGdhD,Armar Y Desarmar,spotify:album:36IP9A6vHdQmDMtCuWCX2N,Al Vuelo,0.975433
spotify:track:3GOLHpXoeasfJA4iiDyFn8,0.639,0.551,7,-8.429,1,0.0334,0.574,0.000016,0.0965,0.506,135.204,287800,Dave Loggins,spotify:track:3GOLHpXoeasfJA4iiDyFn8,spotify:artist:7qBnCBxjX9xHgepd3ayVc5,Breeze,spotify:album:7vCMYLHNDQsoOp0JBldDdH,One Way Ticket To Paradise,0.975297
spotify:track:1760aLXVrLPbLzW6m8CKkI,0.601,0.575,7,-9.098,1,0.0311,0.462,0.003240,0.1390,0.483,137.043,237827,Jordan Klassen,spotify:track:1760aLXVrLPbLzW6m8CKkI,spotify:artist:70QS2gTlkKW2rNjlmMBNp8,Call and Answer,spotify:album:3DHVg26NXUuY6GBUhwGjXu,Monastery,0.975230
spotify:track:3R7Y4q7kECcZkT5LCn8QP5,0.705,0.729,8,-3.925,1,0.0451,0.427,0.000009,0.1500,0.586,126.047,218095,San Holo,spotify:track:3R7Y4q7kECcZkT5LCn8QP5,spotify:artist:0jNDKefhfSbLR9sFvcPLHo,I Still See Your Face,spotify:album:5uR49szrEugTZC16r6sssT,I Still See Your Face,0.975220
spotify:track:0FkD5vWb8VfgOPXAAy7fkA,0.652,0.514,7,-8.363,1,0.1460,0.447,0.000000,0.1080,0.571,132.051,216347,Kyle Thornton & The Company,spotify:track:0FkD5vWb8VfgOPXAAy7fkA,spotify:artist:4mGPVJbOURnByVYcaQZa2J,Lemonade,spotify:album:2HOOaqCgwFCCsMRaPRbwLK,"Space to Move, Pt. I",0.975079


## Evaluation

In the following, we denote the ground truth set of tracks by $G$ 
and the ordered list of recommended tracks by $R$
. The size of a set or list is denoted by $|·|$

$$\text{R-precision} = \frac{\left| G \cap R_{1:|G|} \right|}{|G|} $$

In [25]:
a = ['a','b']
b = ['b','c','d']

In [26]:
list(set(a) & set(b))

['b']

In [27]:
def r_precision(G,R):
    intersect = 0
    for i in list(R):
        if i in list(G):
           intersect += 1
    return intersect/len(list(G))

In [28]:
df_playlists_test

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
6252,Gym,false,6252,1486598400,30,28,1,"[spotify:track:3CzOoYfw7mj5jMHGsJp9TG, spotify...",10,6771523,19,NaN
4684,BOSSanova,false,4684,1498176000,20,13,1,"[spotify:track:0wrx7YMUJXIcThebT8XDz5, spotify...",5,4473182,10,NaN
1731,Disney,false,1731,1502496000,55,27,1,"[spotify:track:4T9Jpft8jvZAP57aE3VdCt, spotify...",3,10028526,39,NaN
4742,Trap,false,4742,1487980800,30,30,2,"[spotify:track:5c1Miz53kZbsJFoa6RvS99, spotify...",4,6434496,27,NaN
4521,Jesus,false,4521,1482537600,80,53,1,"[spotify:track:1wRZLcT8k5oQrf34dRBQb4, spotify...",16,24548027,43,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6412,driving,false,6412,1469577600,45,41,1,"[spotify:track:3KD51YetMdgdPRSF42E35w, spotify...",4,10615480,37,NaN
8285,Smooth rock,false,8285,1502841600,41,22,1,"[spotify:track:48UPSzbZjgc449aqz8bxox, spotify...",5,10711921,14,NaN
7853,heavy,false,7853,1440374400,17,16,1,"[spotify:track:2yuSLHwPoOoB1zDuXnQnqZ, spotify...",4,5568780,7,NaN
1095,Pop en español,false,1095,1485820800,86,53,1,"[spotify:track:5vNocq26ujTMI4L8bECuGk, spotify...",43,20344685,32,NaN


In [29]:
df_playlists_scores_test

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
6252,0.077246,0.543290,0.789533,0.457576,0.872237,0.600000,0.098034,0.064792,0.002102,0.240347,...,0.000000,0.200000,0.733333,0.066667,0.366667,0.633333,0.0,0.100000,0.633333,0.266667
4684,0.523926,0.635318,0.587550,0.495455,0.794836,0.500000,0.054491,0.620181,0.065425,0.186750,...,0.000000,0.000000,1.000000,0.000000,0.100000,0.900000,0.0,0.000000,0.900000,0.100000
1731,0.121074,0.589230,0.537744,0.449587,0.792178,0.727273,0.091453,0.478968,0.004587,0.215700,...,0.000000,0.109091,0.763636,0.127273,0.200000,0.800000,0.0,0.054545,0.600000,0.345455
4742,0.362254,0.573697,0.772233,0.500000,0.873247,0.333333,0.100571,0.079371,0.036983,0.165797,...,0.000000,0.066667,0.633333,0.300000,0.366667,0.633333,0.0,0.100000,0.566667,0.333333
4521,0.263547,0.441662,0.602750,0.418182,0.843779,0.887500,0.038797,0.182700,0.006836,0.181934,...,0.000000,0.037500,0.650000,0.312500,0.237500,0.762500,0.0,0.000000,0.625000,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6412,0.310412,0.534612,0.695822,0.503030,0.849440,0.600000,0.071358,0.217239,0.108603,0.202136,...,0.022222,0.133333,0.711111,0.155556,0.200000,0.800000,0.0,0.044444,0.777778,0.177778
8285,0.490583,0.538308,0.776244,0.405765,0.870631,0.780488,0.047038,0.058120,0.009955,0.171754,...,0.000000,0.048780,0.878049,0.073171,0.146341,0.853659,0.0,0.048780,0.804878,0.146341
7853,0.384468,0.477771,0.885059,0.433155,0.848581,0.411765,0.095474,0.001254,0.145258,0.182135,...,0.000000,0.058824,0.764706,0.176471,0.294118,0.705882,0.0,0.176471,0.705882,0.117647
1095,0.175963,0.555511,0.568151,0.470402,0.854520,0.860465,0.047356,0.368462,0.000055,0.154480,...,0.000000,0.000000,0.976744,0.023256,0.116279,0.883721,0.0,0.000000,0.918605,0.081395


In [30]:
df_track_info_with_artist_album

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_name,track_uri,artist_uri,track_name,album_uri,album_name
uri,,,,,,,,,,,,,,,,,,
spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,125.461,226864,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,The Cookbook
spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,143.040,198800,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,In The Zone
spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,99.259,235933,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,Dangerously In Love (Alben für die Ewigkeit)
spotify:track:1AWQoqb9bSvzTjaLralEkT,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,100.972,267267,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justified
spotify:track:1lzr43nnXAijIGYnCT8M8H,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,94.759,227600,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Hot Shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:4cKPzfOUIZWXhu3GDnESjL,0.551,0.822,11,-7.018,0,0.0614,0.001540,0.000000,0.2150,0.360,127.043,265039,NateWantsToBattle,spotify:track:4cKPzfOUIZWXhu3GDnESjL,spotify:artist:0Vb15td3iKkAzdGD5Sj9Ky,Reluctant Heroes,spotify:album:05XnUTo4HrVy5eLzqcD0lg,Reluctant Heroes
spotify:track:0VICzaRdctD8XGfDstBjQY,0.480,0.761,6,-4.628,1,0.0367,0.120000,0.000000,0.2960,0.359,160.020,368767,Dima Lancaster,spotify:track:0VICzaRdctD8XGfDstBjQY,spotify:artist:7tSqWbPIUCl6bTbPoxLXGn,Your Lie in April - Medley,spotify:album:4CKBpgzwIL7NPJrOvltCHj,Your Lie in April - Medley
spotify:track:0B4YuevrDNhh71edZrFiOs,0.402,0.767,11,-6.718,0,0.1710,0.000077,0.000003,0.3930,0.558,178.293,179267,Set It Off,spotify:track:0B4YuevrDNhh71edZrFiOs,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,Uncontainable,spotify:album:0jGFC44cRkjE2b21aMEbIo,Upside Down


In [31]:
df_track_info_with_artist_album.loc[df_playlists_test.loc[6252,'tracks'],:]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_name,track_uri,artist_uri,track_name,album_uri,album_name
uri,,,,,,,,,,,,,,,,,,
spotify:track:3CzOoYfw7mj5jMHGsJp9TG,0.434,0.961,2,-4.247,1,0.0974,0.000084,0.000003,0.0938,0.2740,153.969,176160,Flyleaf,spotify:track:3CzOoYfw7mj5jMHGsJp9TG,spotify:artist:4IliztYDlfMvzQzbx50o60,I'm So Sick,spotify:album:0XvHcGX9MJXt9Cx1Iml7WS,Flyleaf
spotify:track:1NMXkYmP983tmztiArxJEn,0.495,0.917,7,-3.142,1,0.1160,0.072500,0.000000,0.1920,0.5150,95.153,217690,Flyleaf,spotify:track:1NMXkYmP983tmztiArxJEn,spotify:artist:4IliztYDlfMvzQzbx50o60,Set Me On Fire,spotify:album:1EYbyqvfyrxskx1jniA0Gq,Between The Stars
spotify:track:2EUmhX0S12UgITs19Dq1Xg,0.377,0.912,2,-3.398,1,0.0576,0.000731,0.004600,0.8460,0.5260,146.204,139440,Flyleaf,spotify:track:2EUmhX0S12UgITs19Dq1Xg,spotify:artist:4IliztYDlfMvzQzbx50o60,Call You Out,spotify:album:3l4HWoEbjzsrlekeRWXZPE,New Horizons
spotify:track:0DLjcGTmH2NV9AjzecAGT6,0.320,0.910,9,-5.321,1,0.1140,0.000997,0.000000,0.1600,0.2200,165.943,214373,Red,spotify:track:0DLjcGTmH2NV9AjzecAGT6,spotify:artist:01crEa9G3pNpXZ5m7wuHOk,Breathe Into Me,spotify:album:3DrOZBzNxCPvO1a9sQHrnh,End Of Silence
spotify:track:1p1nO35bbi4ZlQgjIA4oa4,0.179,0.912,4,-3.881,0,0.0791,0.001400,0.000294,0.5820,0.2890,182.990,231533,Thirty Seconds To Mars,spotify:track:1p1nO35bbi4ZlQgjIA4oa4,spotify:artist:0RqtSIYZmd4fiBKVFqyIqD,The Kill (Bury Me),spotify:album:2jPmk6qQzuY5AvKkZxMh7h,A Beautiful Lie
spotify:track:587WH6kCBOPalgEBH2Fk9U,0.333,0.877,5,-1.894,0,0.0603,0.003320,0.001010,0.7690,0.2850,175.095,188187,Thirty Seconds To Mars,spotify:track:587WH6kCBOPalgEBH2Fk9U,spotify:artist:0RqtSIYZmd4fiBKVFqyIqD,ATTACK,spotify:album:2jPmk6qQzuY5AvKkZxMh7h,A Beautiful Lie
spotify:track:6ZWn1MVUaBH2HNQobhMYeE,0.299,0.706,2,-5.421,1,0.0687,0.141000,0.000036,0.0703,0.2470,94.263,204693,The Black Keys,spotify:track:6ZWn1MVUaBH2HNQobhMYeE,spotify:artist:7mnBLXK823vNxN3UWB7Gfz,Little Black Submarines - radio edit,spotify:album:2ZO3hTVFwvi08797IONxbs,Little Black Submarines
spotify:track:6ndmKwWqMozN2tcZqzCX4K,0.688,0.835,8,-4.162,1,0.0911,0.058300,0.000003,0.0795,0.8800,84.858,230493,Fort Minor,spotify:track:6ndmKwWqMozN2tcZqzCX4K,spotify:artist:7dWYWUbO68rXJOcyA7SpJk,Remember The Name (feat. Styles Of Beyond),spotify:album:5v4Vx9loqMQCS3J7OmP9pa,The Rising Tied
spotify:track:6yr8GiTHWvFfi4o6Q5ebdT,0.551,0.851,1,-3.277,1,0.1970,0.074800,0.000000,0.0784,0.1000,171.411,297893,Eminem,spotify:track:6yr8GiTHWvFfi4o6Q5ebdT,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR,'Till I Collapse,spotify:album:1ftvBBcu7jYIvXyt3JWB8S,The Eminem Show


In [38]:
r_precision_track_train = []
r_precision_artist_train = []
for pid in tqdm(df_playlists_train['pid']):
    playlist_feature = df_playlists_scores_train.loc[pid,:]
    num_tracks = df_playlists_train.loc[pid,'num_albums']
    df_recommend = recommend_track_by_feature(playlist_feature,df_track_info_with_artist_album,num_tracks)

    df_original_tracks = df_track_info_with_artist_album.loc[df_playlists_train.loc[pid,'tracks'],:]
    original_tracks = df_original_tracks['track_uri']
    origianl_artists = df_original_tracks['artist_name']
    df_recommend_tracks = df_track_info_with_artist_album.loc[df_recommend['track_uri'],:]
    recommend_tracks = df_recommend_tracks['track_uri']
    recommend_artists = df_recommend_tracks['artist_name']

    r_precision_track_train.append(r_precision(original_tracks,recommend_tracks))
    r_precision_artist_train.append(r_precision(origianl_artists,recommend_artists))


100%|██████████| 8000/8000 [15:13<00:00,  8.76it/s]


In [39]:
print("Evaluation on Train data")
print("Average R-precision of Track:",np.average(r_precision_track_train))
print("Average R-precision of Artist:",np.average(r_precision_artist_train))


Evaluation on Train data
Average R-precision of Track: 0.004025987263481936
Average R-precision of Artist: 0.027837338158265768


Evaluate both the track precision and artist precision

In [36]:
r_precision_track_test = []
r_precision_artist_test = []
for pid in tqdm(df_playlists_test['pid']):
    playlist_feature = df_playlists_scores_test.loc[pid,:]
    num_tracks = df_playlists_test.loc[pid,'num_albums']
    df_recommend = recommend_track_by_feature(playlist_feature,df_track_info_with_artist_album,num_tracks)

    df_original_tracks = df_track_info_with_artist_album.loc[df_playlists_test.loc[pid,'tracks'],:]
    original_tracks = df_original_tracks['track_uri']
    origianl_artists = df_original_tracks['artist_name']
    df_recommend_tracks = df_track_info_with_artist_album.loc[df_recommend['track_uri'],:]
    recommend_tracks = df_recommend_tracks['track_uri']
    recommend_artists = df_recommend_tracks['artist_name']

    r_precision_track_test.append(r_precision(original_tracks,recommend_tracks))
    r_precision_artist_test.append(r_precision(origianl_artists,recommend_artists))

100%|██████████| 2000/2000 [03:23<00:00,  9.85it/s]


In [37]:
print("Evaluation on Test data")
print("Average R-precision of Track:",np.average(r_precision_track_test))
print("Average R-precision of Artist:",np.average(r_precision_artist_test))


Evaluation on Test data
Average R-precision of Track: 0.004407422434661886
Average R-precision of Artist: 0.028705397173856034


In [ ]:
# def recommend_track_by_feature(features,df_track_info,n=10):
#     df_recommendation = df_track_info.copy()
#     df_recommendation['cos_sim'] = cosine_similarity(df_track_info_complete,features.values.reshape(1,-1))
#     return df_recommendation.sort_values('cos_sim',ascending=False).head(n)